In [1]:
import pyclops
from pyclops.core.loss_handler.chemical_loss_handler import ChemicalLossHandler

In [3]:
chem_handler = ChemicalLossHandler.from_pdb_file(pdb_file="/Users/bendupontjr/mphil_files/all_mphil_code/pyclops/examples/pdbs/CysAla5Cys.pdb", 
                                                 units_factor=1.0,
                                                 )

chem_handler._full_summary()

'AmideHead2Tail [0, 6]| AmideHead2Tail [0, 6]| CysCTerm [0, 6]| CysCTerm [0, 6]| Disulfide [0, 6]|'